### Experiment 1 - Logistic Regression

In [30]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score, f1_score, precision_score, recall_score

import mlflow
import dagshub
import logging

print('Setup Complete')

Setup Complete


In [ ]:
# Load data 

df = pd.read_csv('../data/data.csv')
print(df.shape)
df.head()

(6362620, 10)


,step,amount,isFraud,isFlaggedFraud,log_amount,is_high_amount,hour,balance_diff_orig,balance_diff_dest,type_enc
0,1,9839.64,0,0,9.194276,0,1,9839.64,0.0,3
1,1,1864.28,0,0,7.531166,0,1,1864.28,0.0,3
2,1,181.00,1,0,5.204007,0,1,181.00,0.0,4
3,1,181.00,1,0,5.204007,0,1,181.00,-21182.0,1
4,1,11668.14,0,0,9.364703,0,1,11668.14,0.0,3


In [25]:
# Train-test split

X = df.drop(columns=['isFraud','isFlaggedFraud'])
y = df['isFraud'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [26]:
# Experiment tracking

mlflow.set_tracking_uri('https://dagshub.com/jayast29/ml-fraud-detection.mlflow')
dagshub.init(repo_owner='jayast29', repo_name='ml-fraud-detection', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

2026-02-20 22:08:02,050 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/jayast29/ml-fraud-detection "HTTP/1.1 200 OK"


Initialized MLflow to track repo "jayast29/ml-fraud-detection"

2026-02-20 22:08:02,053 - INFO - Initialized MLflow to track repo "jayast29/ml-fraud-detection"


Repository jayast29/ml-fraud-detection initialized!

2026-02-20 22:08:02,053 - INFO - Repository jayast29/ml-fraud-detection initialized!


<Experiment: artifact_location='mlflow-artifacts:/897be242a67b4330acbc652d8e5b22a2', creation_time=1771649856269, experiment_id='0', last_update_time=1771649856269, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}, workspace='default'>

In [ ]:
# MLflow

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logging.getLogger("mlflow").setLevel(logging.ERROR)
logging.getLogger("urllib3").setLevel(logging.ERROR)
logger = logging.getLogger(__name__)

with mlflow.start_run(run_name="logistic_regression_v1"):
    
    logger.info("Starting MLflow run...")
    
    # Scale
    scaler = RobustScaler()
    num_cols = X_train.select_dtypes(include=[np.number]).columns
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols])
    logger.info("Scaling complete")
    
    # Train
    model = LogisticRegression(max_iter=1000, solver='lbfgs', class_weight='balanced', random_state=42)
    model.fit(X_train, y_train)
    logger.info("Model training complete")
    
    # Evaluate
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    roc = roc_auc_score(y_test, y_prob)
    ap = average_precision_score(y_test, y_prob)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log metrics
    mlflow.log_metric("roc_auc", roc)
    mlflow.log_metric("average_precision", ap)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log model
    mlflow.sklearn.log_model(model, "model")
    logger.info("MLflow run complete")
    
    print(classification_report(y_test, y_pred))
    print("ROC-AUC:", roc)
    print("Average Precision:", ap)

2026-02-20 22:13:48,838 - INFO - Starting MLflow run...
2026-02-20 22:13:50,976 - INFO - Scaling complete
2026-02-20 22:14:05,024 - INFO - Model training complete
2026/02/20 22:14:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/20 22:14:10 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html
2026-02-20 22:14:17,086 - INFO - MLflow run complete


              precision    recall  f1-score   support

           0       1.00      0.96      0.98   1270881
           1       0.03      0.88      0.05      1643

    accuracy                           0.96   1272524
   macro avg       0.51      0.92      0.52   1272524
weighted avg       1.00      0.96      0.98   1272524

ROC-AUC: 0.9759934784324132
Average Precision: 0.5412195261575412
🏃 View run logistic_regression_v1 at: https://dagshub.com/jayast29/ml-fraud-detection.mlflow/#/experiments/0/runs/f326dc4222654d348255f949a4806391
🧪 View experiment at: https://dagshub.com/jayast29/ml-fraud-detection.mlflow/#/experiments/0


Logistic Regression achieves a recall of 0.88, catching the majority of fraud cases, but precision remains critically low at 0.03, generating excessive false alarms. ROC-AUC of 0.98 is strong but AUPRC of 0.54 reflects poor precision-recall balance. Not suitable for production due to high false positive rate.